### Import Packages

In [1]:
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

### Load Data

In [2]:
train = pd.read_csv('/nfsroot/data/home/2359B48/Santander Competition/datasets/train.csv')
test = pd.read_csv('/nfsroot/data/home/2359B48/Santander Competition/datasets/test.csv')

In [3]:
print(train.shape)
test.shape

(200000, 202)


(200000, 201)

In [4]:
train.head(3)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965


In [5]:
train.target.value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [6]:
trainX, valX, trainY, valY = train_test_split(train.iloc[:, 2:], train['target'], test_size=0.3, random_state=22)
print(trainX.shape)

(140000, 200)


In [7]:
# param_grid = {
#     'bootstrap':[True],
#     'max_depth': [20, 30, 40],
#     'min_samples_split': [8, 10, 12],
#     'n_estimators': [10, 20, 30]
# }

In [8]:
# rfc = RandomForestClassifier()
# rfc_grid = GridSearchCV(estimator = rfc, param_grid= param_grid, 
#                          cv = 5, verbose=-1, n_jobs = -1)

# rfc_grid.fit(trainX, trainY)

In [ ]:
{'bootstrap': True,
 'max_depth': 40,
 'min_samples_split': 12,
 'n_estimators': 10}

In [9]:
rfc1 = RandomForestClassifier(bootstrap=True, max_depth=40, min_samples_split=12, n_estimators=10)
rfc1.fit(trainX, trainY)
y_pred = rfc1.predict(valX)

train_accuracy = accuracy_score(trainY, rfc1.predict(trainX))
val_accuracy = accuracy_score(valY, y_pred)

In [10]:
train_accuracy

0.9541857142857143

In [11]:
val_accuracy

0.8994

In [16]:
test.head(2)

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760


In [17]:
# Predict on test data

test_pred = rfc1.predict(test.iloc[:, 1:])

In [15]:
test_predictions = pd.DataFrame()
test_predictions[['ID_code']] = test[['ID_code']] 
test_predictions.shape

(200000, 1)

In [20]:
df = pd.DataFrame({'target':test_pred})
df.shape

(200000, 1)

In [22]:
submission = pd.concat([test_predictions, df], axis=1)
submission.shape

(200000, 2)

In [24]:
submission.to_csv('/nfsroot/data/home/2359B48/Santander Competition/submissions/submission1.csv', index=False)

In [ ]:
############################----------------------------------------------------------#########################################

In [7]:
train = train[train.columns[2:]]
test = test[test.columns[1:]]

In [8]:
features = train.columns
train['target'] = 0
test['target'] = 1

In [9]:
train_test = pd.concat([train, test], axis =0)

target = train_test['target'].values

In [10]:
param = {'num_leaves': 50,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 5,
         'learning_rate': 0.006,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 27,
         "metric": 'auc',
         "verbosity": -1}

folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train_test))

In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_test.values, target)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_test.iloc[trn_idx][features], label=target[trn_idx])
    val_data = lgb.Dataset(train_test.iloc[val_idx][features], label=target[val_idx])

    num_round = 30000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 1400)
    oof[val_idx] = clf.predict(train_test.iloc[val_idx][features], num_iteration=clf.best_iteration)

fold n°0
Training until validation scores don't improve for 1400 rounds.
[1000]	training's auc: 0.684197	valid_1's auc: 0.514842
[2000]	training's auc: 0.738172	valid_1's auc: 0.516935
[3000]	training's auc: 0.776343	valid_1's auc: 0.518329
[4000]	training's auc: 0.805587	valid_1's auc: 0.51947
[5000]	training's auc: 0.829721	valid_1's auc: 0.521359
[6000]	training's auc: 0.850241	valid_1's auc: 0.522472
[7000]	training's auc: 0.868758	valid_1's auc: 0.523449
[8000]	training's auc: 0.884855	valid_1's auc: 0.524292
[9000]	training's auc: 0.8985	valid_1's auc: 0.524906
[10000]	training's auc: 0.911029	valid_1's auc: 0.52568
[11000]	training's auc: 0.921936	valid_1's auc: 0.526432
[12000]	training's auc: 0.931455	valid_1's auc: 0.527134
[13000]	training's auc: 0.939714	valid_1's auc: 0.527802
[14000]	training's auc: 0.946954	valid_1's auc: 0.52839
[15000]	training's auc: 0.953304	valid_1's auc: 0.52887
[16000]	training's auc: 0.958937	valid_1's auc: 0.529351
[17000]	training's auc: 0.9639